In [89]:
def ckeck_exist(info,ress):
    for item in ress:
        if info in item['中文名稱']:
            return 0

In [90]:
import requests
from bs4 import BeautifulSoup

ress = []
for genre in range(1, 6):
    for page in range(1,6):
        url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id={}&page={}'.format(genre,page)
        #print(url)
        r = requests.get(url)
        soup = BeautifulSoup(r.text)

        infos = []
        for d in soup.find_all('div', class_="release_info"):
            info = {}
            info['中文名稱'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').text.strip()
            info['英文名稱'] = d.find('div', class_='release_movie_name').find('div', class_='en').a.text.strip()
            info['上映日期'] = d.find('div', class_='release_movie_time').text.split('上映日期 ： ')[1]

            tf = ckeck_exist(info['中文名稱'],ress)
            if tf == 0:
                continue

            infos.append(info)

        for inf in list(infos):
            ress.append(dict(list(inf.items())))


In [91]:
import pandas as pd

df = pd.DataFrame(ress)
df.to_csv('movies.csv', encoding='utf_8_sig')

df

,中文名稱,英文名稱,上映日期
0,小小兵2：格魯的崛起,Minions: The Rise of Guru,2022-07-01
1,蝙蝠俠,The Batman,2022-03-04
2,魔比斯,Morbius,2022-01-16
3,捍衛戰士：獨行俠,Top Gun: Maverick,2021-11-19
4,魔鬼剋星：未來世,Ghostbusters: Afterlife,2021-11-11
...,...,...,...
192,如果雨之後,Waiting For Rain,2021-05-07
193,操控,Every Breath You Take,2021-05-07
194,人生檔案求刪除,Delete History,2021-05-07
195,布吉闖籃關,Boogie,2021-05-07
